## read data from files

In [3]:
import os
import wget
if os.path.isfile("utilsLaris.py")==True:
    os. remove("utilsLaris.py")
wget.download("https://raw.githubusercontent.com/Ah-essabar/LARIS01/master/utilsLaris.py")

'utilsLaris.py'

In [1]:
import janitor
import os
import glob
#import mitosheet
import ipywidgets as widgets
%matplotlib inline
#%matplotlib widget
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import utilsLaris

from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from pandas.plotting import register_matplotlib_converters

from sklearn.model_selection import cross_val_score


%config InlineBackend.figure_format='retina'

register_matplotlib_converters()
sns.set(style='whitegrid', palette='muted', font_scale=1.5)

rcParams['figure.figsize'] = 22, 10

RANDOM_SEED = 42

In [2]:
import pandas as pd

#X=pd.read_csv("X_selected.csv", parse_dates=["date"], index_col="date")
y=pd.read_csv("y_selected.csv", parse_dates=["date"], index_col="date")

In [3]:
#Timedelta('76 days 23:55:00')
def ts_train_test_split(X,y,test_size=0.35,random_state=42, shuffle=True):
    Xtemp, ytemp = X.copy(), y.copy()
    a= X.index - X.index[0]
    a= a.days
    Xtemp['id']=a
    ytemp['id']=a
    from sklearn.model_selection import train_test_split
    indices_train, indices_test=train_test_split(a.unique(), test_size=test_size,random_state=random_state, shuffle=shuffle)
    X_train = Xtemp[Xtemp['id'].isin(indices_train.values)]
    X_test = Xtemp[Xtemp['id'].isin(indices_test.values)]
    y_train = ytemp[ytemp['id'].isin(indices_train.values)]
    y_test = ytemp[ytemp['id'].isin(indices_test.values)]
    X_train = X_train.drop(["id"], axis=1)
    X_test = X_test.drop(["id"], axis=1)
    y_train = y_train.drop(["id"], axis=1)
    y_test = y_test.drop(["id"], axis=1)
    
    return X_train, X_test, y_train, y_test

## Performance final task Random forest regressor
Target varible Electric power

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

import time

start = time.time()

import os
directory = "ImputedData"
print("BDD",";","R_squared_train",";","R_squared_test",";","mse",";" ," rmse",";","R_squared_trainFE",";","R_squared_testFE",";","mseFE",";" ," rmseFE")
for filename in os.listdir(directory):   
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        x = filename
        x = '.'.join(x.split('.')[:-1])
        X=pd.read_csv(f, parse_dates=["date"], index_col="date")        
        X_train, X_test, y_train, y_test = ts_train_test_split(X,y,test_size=0.35,random_state=15, shuffle=True)
        rfr = RandomForestRegressor(n_estimators=300)
        #y_train=y_train.values.ravel()
        rfr.fit(X_train, y_train.values.ravel())
        score = rfr.score(X_train, y_train.values.ravel())
        # print("R-squared:", score) 
        y_pred = rfr.predict(X_test)
        R_squared = r2_score(y_test.values.ravel(), y_pred)
        mse = mean_squared_error(y_test, y_pred)
        rmse = mean_squared_error(y_test, y_pred,squared=False)
        
        X_selectedFeatured, y_selectedFeatured =utilsLaris.FeatureExtraction(X,y, period = '30T')
        X_train, X_test, y_train, y_test = ts_train_test_split(X_selectedFeatured,y_selectedFeatured,test_size=0.35,random_state=15, shuffle=True)
        rfr = RandomForestRegressor(n_estimators=300)
        #y_train=y_train.values.ravel()
        rfr.fit(X_train, y_train.values.ravel())
        scoreFE = rfr.score(X_train, y_train.values.ravel())
        # print("R-squared:", score) 
        y_pred = rfr.predict(X_test)
        R_squaredFE = r2_score(y_test.values.ravel(), y_pred)
        mseFE = mean_squared_error(y_test, y_pred)
        rmseFE = mean_squared_error(y_test, y_pred,squared=False)
        print(x,";",score,";",R_squared,";", mse,";",rmse,";",scoreFE,";",R_squaredFE,";", mseFE,";",rmseFE)


end = time.time()
print(end - start)

BDD ; R_squared_train ; R_squared_test ; mse ;  rmse ; R_squared_trainFE ; R_squared_testFE ; mseFE ;  rmseFE
A_DataComplet ; 0.9887799850527157 ; 0.4085460653576871 ; 13588.703878477181 ; 116.57059611444552 ; 0.9601323939469901 ; 0.5705930517812526 ; 9387.168088929902 ; 96.88739902035714
Back_fillMAR05 ; 0.9899289317498547 ; 0.4212887021602516 ; 13295.940726523933 ; 115.30802542114722 ; 0.9652176769829172 ; 0.5743343872768834 ; 9305.379600596747 ; 96.4643955073412


## Performance mean final task with 5 seed Random forest regressor
Target varible Electric power

In [22]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
import random
import statistics

import time

start = time.time()
fichier1 = open("resultatMean2.txt", "a")
fichier2 = open("resultatStdev2.txt", "a")
import os
directory = "ImputedData"
print("BDD",";","R_squared_train",";","R_squared_test",";","mse",";" ," rmse",";","R_squared_trainFE",";","R_squared_testFE",";","mseFE",";" ," rmseFE")

for filename in os.listdir(directory):   
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):     
        
        x = filename
        x = '.'.join(x.split('.')[:-1])
        X=pd.read_csv(f, parse_dates=["date"], index_col="date")
        score_,R_squared_,mse_,rmse_,scoreFE_,R_squaredFE_, mseFE_,rmseFE_ = [],[],[],[],[],[],[],[]
        for i in range(5):
            seed = random.randint(1, 100)
            X_train, X_test, y_train, y_test = ts_train_test_split(X,y,test_size=0.35,random_state=seed, shuffle=True)
            rfr = RandomForestRegressor(n_estimators=300)
            #y_train=y_train.values.ravel()
            rfr.fit(X_train, y_train.values.ravel())
            score = rfr.score(X_train, y_train.values.ravel())
            # print("R-squared:", score) 
            y_pred = rfr.predict(X_test)
            R_squared = r2_score(y_test.values.ravel(), y_pred)
            mse = mean_squared_error(y_test, y_pred)
            rmse = mean_squared_error(y_test, y_pred,squared=False)

            X_selectedFeatured, y_selectedFeatured =utilsLaris.FeatureExtraction(X,y, period = '30T')
            X_train, X_test, y_train, y_test = ts_train_test_split(X_selectedFeatured,y_selectedFeatured,test_size=0.35,random_state=seed, shuffle=True)
            rfr = RandomForestRegressor(n_estimators=300)
            #y_train=y_train.values.ravel()
            rfr.fit(X_train, y_train.values.ravel())
            scoreFE = rfr.score(X_train, y_train.values.ravel())
            # print("R-squared:", score) 
            y_pred = rfr.predict(X_test)
            R_squaredFE = r2_score(y_test.values.ravel(), y_pred)
            mseFE = mean_squared_error(y_test, y_pred)
            rmseFE = mean_squared_error(y_test, y_pred,squared=False)
            #print(x,";",score_,";",R_squared_,";", mse_,";",rmse_,";",scoreFE_,";",R_squaredFE_,";", mseFE_,";",rmseFE_)
            score_.append(score),R_squared_.append(R_squared),mse_.append(mse),rmse_.append(rmse),scoreFE_.append(scoreFE),R_squaredFE_.append(R_squaredFE), mseFE_.append(mseFE),rmseFE_.append(rmseFE)

        lignemean = str(x)+";"+str(statistics.mean(score_))+";"+str(statistics.mean(R_squared_))+";"+ str(statistics.mean(mse_))+";"+str(statistics.mean(rmse_))+";"+str(statistics.mean(scoreFE_))+";"+str(statistics.mean(R_squaredFE_))+";"+ str(statistics.mean(mseFE_))+";"+str(statistics.mean(rmseFE_))
        lignestdev = str(x)+";"+str(statistics.stdev(score_))+";"+str(statistics.stdev(R_squared_))+";"+ str(statistics.stdev(mse_))+";"+str(statistics.stdev(rmse_))+";"+str(statistics.stdev(scoreFE_))+";"+str(statistics.stdev(R_squaredFE_))+";"+ str(statistics.stdev(mseFE_))+";"+str(statistics.stdev(rmseFE_))
        fichier1 = open("resultatMean2.txt", "a")
        fichier2 = open("resultatStdev2.txt", "a")
        print(lignemean)
        fichier1.write(lignemean)
        fichier2.write(lignestdev)
        fichier1.close()
        fichier2.close()
        # print(i)

    end = time.time()
    #print(end - start)

BDD ; R_squared_train ; R_squared_test ; mse ;  rmse ; R_squared_trainFE ; R_squared_testFE ; mseFE ;  rmseFE


KeyboardInterrupt: 

In [46]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
X=pd.read_csv("./ImputedData/A_dataComplet.csv", parse_dates=["date"], index_col="date")        
X_train, X_test, y_train, y_test = ts_train_test_split(X,y,test_size=0.15,random_state=0, shuffle=True)
rfr = RandomForestRegressor(n_estimators=100, max_depth=7,max_features=0.7)
#y_train=y_train.values.ravel()
rfr.fit(X_train, y_train.values.ravel())
score = rfr.score(X_train, y_train.values.ravel())
# print("R-squared:", score) 
y_predTest = rfr.predict(X_test)
y_predTrain = rfr.predict(X_train)
rmseTest = mean_squared_error(y_test, y_predTest,squared=False)
rmseTrain = mean_squared_error(y_train, y_predTrain,squared=False)
print(rmseTrain,";",rmseTest)

54.65862289217296 ; 56.710529491275295


## deal with overfeting

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn import svm
from sklearn.svm import SVR
import random
import time

start = time.time()

import os
directory = "ImputedData"
print("BDD",";mean_std;","score",";","rmseTrain",";"," rmseTest",";","rmseTrainFE",";","rmseTestFE")
# seed = 30
test_size = 0.20

for filename in os.listdir(directory):   
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        x = filename
        x = '.'.join(x.split('.')[:-1])
        X=pd.read_csv(f, parse_dates=["date"], index_col="date")        
        # X=pd.read_csv("./ImputedData/A_dataComplet.csv", parse_dates=["date"], index_col="date")
        score_,rmseTrain_,rmseTest_,rmseTrainFE_,rmseTestFE_ = [],[],[],[],[]
        for i in range(10):
            random.seed(3**i) 
            seed = random.randint(0, 100)
            # print(seed)
            X_train, X_test, y_train, y_test = ts_train_test_split(X,y,test_size=test_size, shuffle=True)
            rfr = RandomForestRegressor(n_estimators = 100,oob_score = True,n_jobs = -1, max_depth=5,max_features=0.33,min_samples_leaf=100)      

            # rfr = make_pipeline(StandardScaler(), SVR(C=1.0, epsilon=0.2))

            #y_train=y_train.values.ravel()
            rfr.fit(X_train, y_train.values.ravel())
            score = rfr.score(X_train, y_train.values.ravel())
            # print("R-squared:", score) 
            y_predTest = rfr.predict(X_test)
            y_predTrain = rfr.predict(X_train)
            rmseTest = mean_squared_error(y_test, y_predTest,squared=False)
            rmseTrain = mean_squared_error(y_train, y_predTrain,squared=False)


            X_selectedFeatured, y_selectedFeatured =utilsLaris.FeatureExtraction(X,y, period = '30T')
            X_train, X_test, y_train, y_test = ts_train_test_split(X_selectedFeatured,y_selectedFeatured,test_size=test_size, shuffle=True)
            rfr = RandomForestRegressor(n_estimators = 100,oob_score = True,n_jobs = -1, max_depth=5,max_features=0.33,min_samples_leaf=100)  
            # rfr = make_pipeline(StandardScaler(), SVR(C=1.0, epsilon=0.2))
            #y_train=y_train.values.ravel()
            rfr.fit(X_train, y_train.values.ravel())
            scoreFE = rfr.score(X_train, y_train.values.ravel())
            # print("R-squared:", score) 
            y_predTest= rfr.predict(X_test)
            y_predTrain= rfr.predict(X_train)
            rmseTestFE = mean_squared_error(y_test, y_predTest,squared=False)
            rmseTrainFE = mean_squared_error(y_train, y_predTrain,squared=False)
            score_.append(score)
            rmseTrain_.append(rmseTrain)
            rmseTest_.append(rmseTest)
            rmseTrainFE_.append(rmseTrainFE)
            rmseTestFE_.append(rmseTestFE)


        print(x,";mean;",np.mean(score_),";",np.mean(rmseTrain_),";", np.mean(rmseTest_),";",np.mean(rmseTrainFE_),";",np.mean(rmseTestFE_))
        print(x,";std;",np.std(score_),";",np.std(rmseTrain_),";", np.std(rmseTest_),";",np.std(rmseTrainFE_),";",np.std(rmseTestFE_))

end = time.time()
print(end - start)

BDD ;mean_std; score ; rmseTrain ;  rmseTest ; rmseTrainFE ; rmseTestFE
A_DataComplet ;mean; 0.5528373208043258 ; 89.30120186733525 ; 65.38191885639057 ; 94.95505565387721 ; 59.495297743696746
A_DataComplet ;std; 0.0026688012584782286 ; 0.2665367067241156 ; 0.45639434483374874 ; 0.3134029518810972 ; 0.23095373392342552
Back_fillMAR05 ;mean; 0.5689993016047311 ; 87.67251596376889 ; 63.75769070239208 ; 94.98174939811827 ; 59.11733742587741
Back_fillMAR05 ;std; 0.002600171573963342 ; 0.26478605006942174 ; 0.3771404198713547 ; 0.2610961857793185 ; 0.24357889976943523
Back_fillMAR10 ;mean; 0.5552544231940206 ; 89.05967610404491 ; 64.97977513107115 ; 94.76777723406444 ; 59.06554177694702
Back_fillMAR10 ;std; 0.002065354773238272 ; 0.20681297464469675 ; 0.5056242444384816 ; 0.36337377222738226 ; 0.2553357459051073
Back_fillMAR25 ;mean; 0.5629882687457147 ; 88.2817920985377 ; 64.88266620522516 ; 95.47284530377263 ; 59.19584625941182
Back_fillMAR25 ;std; 0.0025641360379391243 ; 0.25911191538991